In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
import cv2

In [4]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5



In [16]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight

In [21]:
#CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks (batch_sizes={"Medium": 32}, epoch_sizes= {"Dev": 5,}, crop={"C0": False}, name_suffix="",)



In [22]:
#CHEXPERT_BENCHMARKS.keys() 

In [23]:
#CHESTXRAY14_BENCHMARKS.keys()

In [24]:
#[m.name for m in METRICS]

In [25]:
#CHEXPERT_COLUMNS

In [26]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('jupyterlab', 'http://64507fb37a13.ngrok.io'), ('tensorboard', 'http://154ac13baf36.ngrok.io')]


In [27]:
#CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"].as_dict()

In [28]:
#chexpert_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"], DenseNet121, CHEXPERT_COLUMNS)

In [29]:
#chexpert_exp.benchmark.as_dict()

In [30]:
#chexpert_exp.run()

In [6]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Lesion',
        'Pneumonia',
        'Atelectasis',
        'Fracture',
        'Pleural Other']

uones = ['Edema',
         'Lung Opacity',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']


upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}

columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros_5 = ['Cardiomegaly',
        'Consolidation',
        'Pleural Effusion']

uones_5 = ['Edema',
        'Atelectasis']


upsample_factors_5 = {
    "Consolidation":2,
    "Cardiomegaly":1
}

transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}

In [17]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                 name="3_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                 classes=columns_12,
                                 train_labels = "train.csv",
                                 test_labels = "test.csv",
                                 nan_replacement = 0, #float("NaN"),
                                 u_enc = [uzeros, uones],
                                 epochs=5,
                                 batch_size=32,
                                 crop = False,
                                 dim=(256, 256),
                                 loss = BinaryCrossentropy(),
                                 use_class_weights = False,
                                 upsample_factors = upsample_factors,
                                 metrics=METRICS,
                                 single_class_metrics=SINGLE_CLASS_METRICS,
                                 optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                 lr_factor = 0.1,
                                 augmentation = "affine",
                                 transformations = transformations_2,
                                 split_seed = 6122156,
                                 split_valid_size = 0.05, 
                                 preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)
bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

Initialzed custom_binary_crossentropy with epsilon 1e-05


In [18]:
bce_benchmark.as_dict()

{'benchmark_name': '3_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 5,
 'optimizer': 'Adam',
 'learning_rate': 0.0001,
 'lr_factor': 0.1,
 'loss': 'weighted_binary_crossentropy',
 'use_class_weights': False,
 'class_weights': [0.8205518126487732,
  0.6745193004608154,
  0.1585094928741455,
  0.9674689173698425,
  0.2945497930049896,
  0.42419853806495667,
  2.972834348678589,
  0.5435968041419983,
  0.7648646235466003,
  0.186068594455719,
  1.5195810794830322,
  0.6732566356658936],
 'positive_weights': [0.8205518126487732,
  0.6745193004608154,
  0.1585094928741455,
  0.9674689173698425,
  0.2945497930049896,
  0.42419853806495667,
  2.972834348678589,
  0.5435968041419983,
  0.7648646235466003,
  0.186068594455719,
  1.5195810794830322,
  0.6732566356658936],
 'negative_weights': [0.7299655079841614,
  0.7472195625305176,


In [19]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet121, bce_benchmark.label_columns)

In [ ]:
bce_chexpert_exp.train()

Epoch 1/5
1229/8101 [===>..........................] - ETA: 35:31 - loss: 0.2344 - auc: 0.6899 - precision: 0.6293 - recall: 0.0329 - f2_score: 0.0406 - binary_accuracy: 0.8260 - accuracy_enlarged_cardiomediastinum: 0.9013 - accuracy_cardiomegaly: 0.8844 - accuracy_lung_opacity: 0.5043 - accuracy_lung_lesion: 0.9176 - accuracy_edema: 0.7441 - accuracy_consolidation: 0.8165 - accuracy_pneumonia: 0.9717 - accuracy_atelectasis: 0.8552 - accuracy_pneumothorax: 0.8921 - accuracy_pleural_effusion: 0.6049 - accuracy_pleural_other: 0.9447 - accuracy_fracture: 0.8757 - auc_enlarged_cardiomediastinum: 0.5980 - auc_cardiomegaly: 0.7516 - auc_lung_opacity: 0.6716 - auc_lung_lesion: 0.6769 - auc_edema: 0.7934 - auc_consolidation: 0.6325 - auc_pneumonia: 0.6443 - auc_atelectasis: 0.6263 - auc_pneumothorax: 0.7037 - auc_pleural_effusion: 0.7909 - auc_pleural_other: 0.7260 - auc_fracture: 0.6638 - precision_enlarged_cardiomediastinum: 0.1639 - precision_cardiomegaly: 0.5545 - precision_lung_opacity: 0

In [ ]:
bce_chexpert_exp.evaluate()

In [ ]:
bce_chexpert_exp.save()

In [ ]:

transformations_4 = {"unsharp_mask":{"radius":5, "amount":1}}

In [ ]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                 name="Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp51_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                 classes=columns_12,
                                 train_labels = "train.csv",
                                 test_labels = "test.csv",
                                 nan_replacement = 0, #float("NaN"),
                                 u_enc = [uzeros, uones],
                                 epochs=5,
                                 batch_size=32,
                                 crop = False,
                                 dim=(256, 256),
                                 loss = BinaryCrossentropy(),
                                 use_class_weights = False,
                                 upsample_factors = upsample_factors,
                                 metrics=METRICS,
                                 single_class_metrics=SINGLE_CLASS_METRICS,
                                 optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                 lr_factor = 0.1,
                                 augmentation = "affine",
                                 transformations = transformations_4,
                                 split_seed = 6122156,
                                 split_valid_size = 0.05, 
                                 preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)
bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

In [ ]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet121, bce_benchmark.label_columns)

In [ ]:
bce_chexpert_exp.train()

In [ ]:
bce_chexpert_exp.evaluate()

In [ ]:
bce_chexpert_exp.save()

In [7]:
transformations_5 = { "unsharp_mask":{"radius":2, "amount":1}}

In [8]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                 name="2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF5_Adam_Upsampled",
                                 classes=columns_12,
                                 train_labels = "train.csv",
                                 test_labels = "test.csv",
                                 nan_replacement = 0, #float("NaN"),
                                 u_enc = [uzeros, uones],
                                 epochs=5,
                                 batch_size=32,
                                 crop = False,
                                 dim=(256, 256),
                                 loss = BinaryCrossentropy(),
                                 use_class_weights = False,
                                 upsample_factors = upsample_factors,
                                 metrics=METRICS,
                                 single_class_metrics=SINGLE_CLASS_METRICS,
                                 optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                 lr_factor = 0.5,
                                 augmentation = "affine",
                                 transformations = transformations_5,
                                 split_seed = 6122156,
                                 split_valid_size = 0.05, 
                                 preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)
bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

Initialzed custom_binary_crossentropy with epsilon 1e-05


In [10]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet169, bce_benchmark.label_columns)


In [11]:
bce_chexpert_exp.model.load_weights("models/DenseNet169_2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF5_Adam_Upsampled/weights.05-0.19.hdf5")

In [35]:
bce_chexpert_exp.train()

Epoch 1/5
8101/8101 [==============================] - ETA: 0s - loss: 0.2269 - auc: 0.7174 - precision: 0.7082 - recall: 0.0608 - f2_score: 0.0742 - binary_accuracy: 0.8295 - accuracy_enlarged_cardiomediastinum: 0.9011 - accuracy_cardiomegaly: 0.8874 - accuracy_lung_opacity: 0.5001 - accuracy_lung_lesion: 0.9180 - accuracy_edema: 0.7512 - accuracy_consolidation: 0.8144 - accuracy_pneumonia: 0.9719 - accuracy_atelectasis: 0.8528 - accuracy_pneumothorax: 0.9001 - accuracy_pleural_effusion: 0.6304 - accuracy_pleural_other: 0.9450 - accuracy_fracture: 0.8820 - auc_enlarged_cardiomediastinum: 0.6282 - auc_cardiomegaly: 0.7835 - auc_lung_opacity: 0.6835 - auc_lung_lesion: 0.7107 - auc_edema: 0.8111 - auc_consolidation: 0.6470 - auc_pneumonia: 0.6682 - auc_atelectasis: 0.6482 - auc_pneumothorax: 0.7406 - auc_pleural_effusion: 0.8155 - auc_pleural_other: 0.7758 - auc_fracture: 0.6964 - precision_enlarged_cardiomediastinum: 0.4633 - precision_cardiomegaly: 0.5906 - precision_lung_opacity: 0.91

In [12]:
bce_chexpert_exp.evaluate()

234/234 [==============================] - 6s 25ms/step


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

234/234 [==============================] - 15s 64ms/step - loss: 0.2894 - auc: 0.6876 - precision: 0.6049 - recall: 0.0897 - f2_score: 0.1082 - binary_accuracy: 0.8116 - accuracy_enlarged_cardiomediastinum: 0.5427 - accuracy_cardiomegaly: 0.7308 - accuracy_lung_opacity: 0.4615 - accuracy_lung_lesion: 0.9872 - accuracy_edema: 0.8718 - accuracy_consolidation: 0.8590 - accuracy_pneumonia: 0.9615 - accuracy_atelectasis: 0.6581 - accuracy_pneumothorax: 0.9402 - accuracy_pleural_effusion: 0.7735 - accuracy_pleural_other: 0.9829 - accuracy_fracture: 0.9701 - auc_enlarged_cardiomediastinum: 0.5261 - auc_cardiomegaly: 0.7503 - auc_lung_opacity: 0.9166 - auc_lung_lesion: 0.1395 - auc_edema: 0.9225 - auc_consolidation: 0.8979 - auc_pneumonia: 0.6153 - auc_atelectasis: 0.7784 - auc_pneumothorax: 0.8841 - auc_pleural_effusion: 0.9130 - auc_pleural_other: 0.9077 - auc_fracture: 0.0000e+00 - precision_enlarged_cardiomediastinum: 1.0000 - precision_cardiomegaly: 1.0000 - precision_lung_opacity: 0.0000

{'report': '                            precision    recall  f1-score   support\n\nEnlarged Cardiomediastinum       1.00      0.02      0.04       109\n              Cardiomegaly       1.00      0.07      0.14        68\n              Lung Opacity       0.00      0.00      0.00       126\n               Lung Lesion       0.00      0.00      0.00         1\n                     Edema       0.83      0.42      0.56        45\n             Consolidation       0.00      0.00      0.00        33\n                 Pneumonia       0.33      0.12      0.18         8\n               Atelectasis       0.00      0.00      0.00        80\n              Pneumothorax       0.25      0.38      0.30         8\n          Pleural Effusion       0.79      0.28      0.42        67\n             Pleural Other       0.00      0.00      0.00         1\n                  Fracture       0.00      0.00      0.00         0\n\n                 micro avg       0.60      0.09      0.16       546\n                 m

In [19]:
bce_chexpert_exp.save()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /srv/idp-radio-1/models/DenseNet169_2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF5_Adam_Upsampled/DenseNet169_2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF5_Adam_Upsampled_20200906-124223/assets


'697b4e66-f03e-11ea-a627-0242ac110002'

NameError: name 'bce_chexpert_exp' is not defined

In [ ]:
compute_class_weight(bce_chexpert_exp.benchmark.traingen)

In [ ]:
trained_weights = bce_chexpert_exp.model.get_weights()

In [ ]:
images, labels = bce_chexpert_exp.benchmark.traingen[0]
len(images)

In [ ]:
preds = bce_chexpert_exp.model(images, training=True)
preds.op

In [ ]:
bce_chexpert_exp.model.summary(line_length=150)

In [ ]:
index = -1
trained_weights[index].shape, trained_weights[index], np.isnan(trained_weights[index]).any()

In [ ]:
nan_layers = [i for i in range(len(trained_weights)) if np.isnan(trained_weights[i]).any()] 
#nan_layers   

In [ ]:
mean_weights = {i:np.abs(trained_weights[i]).mean() for i in range(len(trained_weights))}
#mean_weights

In [ ]:
{i:trained_weights[i].shape for i in range(len(trained_weights))}

In [ ]:
for i in range(191, 195):
    print(i, trained_weights[i].shape, trained_weights[i])

In [ ]:
#chexpert_exp.save()

In [ ]:
from datetime import datetime
model_filename = chexpert_exp.model_name + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_filename_tf = model_filename + ".tf"
model_filename_h5 = model_filename + ".h5"
model_filename

In [ ]:
folderpath = Path(os.getcwd()) / 'models' / chexpert_exp.model_name
path = folderpath / model_filename
path

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(path, save_format="tf")

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(folderpath / model_filename_h5, save_format="h5")

In [ ]:
!pwd

In [ ]:
folderpath / model_filename_h5

In [ ]:
chexpert_exp.save()

In [ ]:
chexpert_exp.evaluate()

In [ ]:
testgen = chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].testgen

In [ ]:
preds = chexpert_exp.model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
import json
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from pathlib import Path
from keras.utils.generic_utils import get_custom_objects
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight


groundtruth_label = testgen.get_labels_nonan()
thresholds = [0.5]
for t in thresholds:
    print('current threshold: ', t)
    predictions_bool = (preds >= t)
    y_pred = np.array(predictions_bool, dtype=int)

    # report = classification_report(groundtruth_label, y_pred, target_names=list(CHEXPERT_BENCHMARKS['WBCE_E5_B32_C0'].label_columns))
    # print('sklearn report: ', report)
    
    pre = Precision(thresholds=t)
    pre.update_state(groundtruth_label, predictions_bool)
    print('TF Precision: ', pre.result())
    pre.reset_states()
    
    rec = Recall(thresholds=t)
    rec.update_state(groundtruth_label, predictions_bool)
    print('TF Recall: ', rec.result())
    rec.reset_states()

In [ ]:
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

In [ ]:
model = tf.keras.models.load_model('models/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7_20200819-125620.h5', compile=False)

In [ ]:
predstwo = model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
groundtruth_label = testgen.get_labels_nonan()
predictions_bool = (predstwo >= t)
y_pred = np.array(predictions_bool, dtype=int)
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

In [ ]:
"test.h5".replace(".h5", "")

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, -1, -1, 1], [-1, 0, -1, -1]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_true

In [ ]:
y_pred = tf.constant(y_pred)
y_pred

In [ ]:
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
     backend_config.epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

In [ ]:
mask = tf.cast(tf.math.greater_equal(y_true, 0), tf.float32)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)

bce_pos = tf.math.multiply_no_nan(bce_pos, mask)
bce_neg = tf.math.multiply_no_nan(bce_neg, mask)

bce = bce_pos * weights_pos + \
    bce_neg * weights_neg

loss = tf.reduce_sum(-bce) / tf.reduce_sum(mask)
loss

In [ ]:
tf.reduce_sum(mask)

In [ ]:
bce_neg

In [ ]:
bce

In [ ]:

from tensorflow.keras import backend as K
